In [51]:
from mesh_to_sdf import mesh_to_sdf

import trimesh
import pyrender
import numpy as np

tfs = np.eye(4)
# Translate anchor (center) to desired location
tfs[:3,3] = np.array([-3, -2, 15])

sm = trimesh.creation.uv_sphere(radius=10)
sm.apply_transform(tfs)
sm.visual.vertex_colors = [0.0, 1.0, 0.0, 0.1]

low_bounds = [-1, -1, -1]
up_bounds = [15, 20, 15]
increment = 0.1

x = np.arange(low_bounds[0], up_bounds[0], increment)
y = np.arange(low_bounds[1], up_bounds[1], increment)
z = np.arange(low_bounds[2], up_bounds[2], increment)

sdf_points = []

for xi in x:
    for yi in y:
        for zi in z:
            sdf_points.append([xi, yi, zi])

sdf_points = np.array(sdf_points)


In [43]:
sdf = mesh_to_sdf(sm, sdf_points)
print(sdf.shape)

(5376000,)


In [52]:
colors = np.zeros(sdf_points.shape)
colors[sdf < 0, 2] = 1
colors[sdf > 0, 0] = 1
cloud = pyrender.Mesh.from_points(sdf_points, colors=colors)
scene = pyrender.Scene()
scene.add(cloud)
viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=5, show_world_axis=True)


In [44]:
sdf_new = np.reshape(sdf, (x.shape[0],y.shape[0],z.shape[0]))
print(sdf_new.shape)
np.save("sphere_sdf.npy", sdf_new)


(160, 210, 160)


In [45]:
# This gives the volume normals
sdf_grad = np.array(np.gradient(sdf_new, x, y, z)) 
sdf_grad = np.moveaxis(sdf_grad, 0, -1)
print(sdf_grad.shape)
np.save("sphere_sdf_grad.npy", sdf_grad)


(160, 210, 160, 3)


In [10]:
def find_nearest_ind(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def compute_ind(low_bound, up_bound, inc, v):
    if v <= low_bound:
        return 0
    elif v >= up_bound:
        return -1
    else:
        shifted_v = v - low_bound
        return np.floor(shifted_v / inc).astype(int)

def get_inds(x, y, z, p):
    inds = [0, 0, 0]
    # inds[0] = find_nearest_ind(x,p[0])
    # inds[1] = find_nearest_ind(y,p[1])
    # inds[2] = find_nearest_ind(z,p[2])
    inds[0] = compute_ind(low_bounds[0], up_bounds[0], increment, p[0])
    inds[1] = compute_ind(low_bounds[1], up_bounds[1], increment, p[1])
    inds[2] = compute_ind(low_bounds[2], up_bounds[2], increment, p[2])
    inds = np.array(inds)
    return inds

def get_val(x, y, z, sdf, p):
    inds = get_inds(x, y, z, p)
    val = sdf[inds[0], inds[1], inds[2]]
    return val

def get_grads(x, y, z, sdf_grad, p):
    ind = get_inds(x, y, z, p)
    grad = [0, 0, 0]
    grad[0] = sdf_grad[ind[0], ind[1], ind[2], 0]
    grad[1] = sdf_grad[ind[0], ind[1], ind[2], 1]
    grad[2] = sdf_grad[ind[0], ind[1], ind[2], 2]
    grad = np.array(grad)
    return grad
